This code measures DHE flourescence; the rate of which signals the rate of cytosolic reactive oxygen species production. Here, we compare the rate of cytosolic ROS produced with the knockdown of TAX1BP1 or ADAMTS19 to the SCR control.

Import Libraries

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from skimage.filters import gaussian
import czifile

Define Sub Functions

In [4]:
def analyze_time_course_image_mask_saturation(image_path):
    """
    Analyze the mean fluorescence intensity over time for an image, applying a mask 
    to exclude saturated and blank areas.

    Parameters:
    image_path (str): Path to the .czi image file.

    Returns:
    intensity_list (list): List of mean intensities over time.
    times (list): List of time points corresponding to each intensity measurement.
    """
    czi_file = czifile.CziFile(image_path)
    czi_data = czi_file.asarray()
    green_channel_data = np.squeeze(czi_data[:, :, 0, :, :, :])
    last_frame = green_channel_data[-1, :, :]
    filtered_frame = gaussian(last_frame)
    last_frame_mask = filtered_frame > 0.02

    intensity_list = []
    times = []
    for frame_index in range(green_channel_data.shape[0]):
        green_channel = green_channel_data[frame_index, :, :]
        times.append(frame_index)
        mean_intensity = np.mean(green_channel[last_frame_mask > 0])
        intensity_list.append(mean_intensity)

    return intensity_list, times


Define Main Function

In [5]:
def main():
    """
    Main function to analyze time-course images of the green channel, plot results, 
    and save the data to an Excel file.
    """
    folder_path = r"test_images"
    all_data = []

    for well_image in os.listdir(folder_path):
        if well_image.lower().endswith(".czi"):  # Filter for CZI files
            well_image_path = os.path.join(folder_path, well_image)
            well_image_base_name = os.path.basename(well_image)[:-4]

            # Analyze the image and calculate the slope of normalized MFI over time
            intensity_list, times = analyze_time_course_image_mask_saturation(well_image_path)

            df = pd.DataFrame({
                "Filename": [well_image_base_name] * len(intensity_list),
                "Time": times,
                "MFI": intensity_list,
            })
            all_data.append(df)

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_csv_path = os.path.join("DHE.xlsx")
    combined_df.to_excel(combined_csv_path, index=False)

# Run the main function
if __name__ == "__main__":
    main()